## Anisimov Anton NLP HW2 (BBC)

In [ ]:
# Short plan(aka pipeline):
#     1)prepare the datasets
#     2)try models such as:log-reg,randomforest,CNN
#     3)compere results
## Metrix: f1 weighted

In [19]:
import glob
import pandas as pd
import numpy as np
# data prep
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
# models
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import xgboost as xgb

## DATA PREPORATION

In [201]:
#LOAD CLASSES PATH and NAMES
path='bbc-fulltext/bbc/'
cat=glob.glob(path+'*')
cat=[cat[i].split('\\')[-1] for i in range(0,len(cat))]
cat

['business', 'entertainment', 'politics', 'sport', 'tech']

In [202]:
#LOAD DATA
text=[]
labels=[]
k=0
for i in cat:
    d=glob.glob('bbc-fulltext/bbc/'+i+'/*.txt')
    for i in d:
        try:
    # block raising an exception
          with open(i, 'r') as file:
                data = file.read().replace('\n', '')
                text.append([data])
                labels.append(k)
        except:
            pass 
    k+=1

In [41]:
#LOAD composed data to pandas
df = pd.DataFrame(data=text,columns=['observation'])
df['final_labels']=labels

In [203]:
df.head()

,Unnamed: 0,observation,final_labels
0,0,Ad sales boost Time Warner profitQuarterly pro...,0
1,1,Dollar gains on Greenspan speechThe dollar has...,0
2,2,Yukos unit buyer faces loan claimThe owners of...,0
3,3,High fuel prices hit BA's profitsBritish Airwa...,0
4,4,Pernod takeover talk lifts DomecqShares in UK ...,0


In [43]:
#  df.to_csv('bbc.csv')

In [5]:
df=pd.read_csv('bbc.csv')
df=df.drop(columns=df.columns[0])
df.head()

,observation,final_labels
0,Ad sales boost Time Warner profitQuarterly pro...,0
1,Dollar gains on Greenspan speechThe dollar has...,0
2,Yukos unit buyer faces loan claimThe owners of...,0
3,High fuel prices hit BA's profitsBritish Airwa...,0
4,Pernod takeover talk lifts DomecqShares in UK ...,0


In [6]:
X,y=df['observation'],df.iloc[:,-1]
#check class balance
y.iloc[:].value_counts().sort_index()

0    510
1    386
2    417
3    511
4    401
Name: final_labels, dtype: int64

## CODE WORDS

 Bags of words set:

In [7]:
# Create dictionary
vectorizer = CountVectorizer()
Vec = vectorizer.fit_transform(X,y=y)
df_bags=pd.DataFrame(data=Vec.toarray(),columns=vectorizer.get_feature_names())
df_bags.head()

,00,000,0001,000a,000bn,000four,000m,000s,000th,001,...,вј952m,вј960m,вј96bn,вј97m,вј98,вј980m,вј98m,вј99,вј9m,ј15
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Tf-idа set:

In [8]:
vectorizer = TfidfVectorizer()
Vec=vectorizer.fit_transform(X,y)
col=vectorizer.get_feature_names()
df_tf=pd.DataFrame(Vec.todense(),columns=col)
df_tf.head()

,00,000,0001,000a,000bn,000four,000m,000s,000th,001,...,вј952m,вј960m,вј96bn,вј97m,вј98,вј980m,вј98m,вј99,вј9m,ј15
0,0.0,0.020955,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.018927,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Experimental cleaning up:

Cheking tresh:

In [9]:
# Select tresh by freq
a=df_bags.sum().sort_values()
a[-200:]

don           444
service       452
during        456
chief         457
growth        458
london        459
own           460
me            465
day           466
come          466
months        466
brown         470
plans         471
put           472
already       473
half          473
old           478
six           481
sales         489
even          496
group         496
four          496
expected      505
where         513
news          513
end           514
both          514
these         514
however       514
tv            517
            ...  
this         2844
we           3004
mr           3005
his          3026
they         3083
not          3484
from         3535
are          4401
but          4419
will         4472
by           4511
at           4634
have         4771
has          4956
as           4974
with         5352
be           5805
he           5939
was          6023
said         7255
on           7619
it           7887
that         8256
is           8554
for       

 About 200 last freq words are the useless(general articles and etc) and there are lot of words which use ones about 25000. However, I'm going to drop them and try to "experimental learning"

In [10]:
# Drop "tresh"
a=list(a[:25000].index) + list(a[-400:].index)
df_cl=df_bags.drop(columns=a)

In [11]:
X_b_train, X_b_test, y_b_train, y_b_test = train_test_split(df_bags.iloc[:,:], y, test_size=0.33, random_state=20)
X_tf_train, X_tf_test, y_tf_train, y_tf_test = train_test_split(df_tf.iloc[:,:], y, test_size=0.33, random_state=20)
X_cl_train, X_cl_test, y_cl_train, y_cl_test = train_test_split(df_cl.iloc[:,:], y, test_size=0.33, random_state=20)

In [12]:
set_mat=[[X_b_train, X_b_test, y_b_train, y_b_test],
        [X_tf_train, X_tf_test, y_tf_train, y_tf_test],
        [X_cl_train, X_cl_test, y_cl_train, y_cl_test]]

In [13]:
#score matrix
final_columns='bags','tfidf,cl_bags'

## MODELS

In [14]:
#define function to save model
def save_model(Model,name):
    import pickle
    a=0
    for i in Model:
        name=name+str(a)+'.sav'
        pickle.dump(i, open(name, 'wb'))
        a+=1

In [15]:
scores=[]

Log-reg.CV:

In [21]:
Models_log=[]
score_log=[]
for i in range(0,3):
    Model1 = LogisticRegression(random_state=0, solver='newton-cg',
                             multi_class='multinomial').fit(set_mat[i][0], set_mat[i][2])
    train_scores = cross_val_score(Model1, set_mat[i][0], set_mat[i][2], cv=5,scoring='f1_weighted').mean()
    test_scores=metrics.f1_score(y_true=set_mat[i][3],y_pred=Model1.predict(set_mat[i][1]),average='weighted')
    print ('train:',train_scores,'test:',test_scores)
    score_log.append(test_scores)
    Models_log.append(Model1)
scores.append(score_log)
# save models
save_model(Models_log,'Models/Log_model')

train: 0.963600540940382 test: 0.9632187302100251
train: 0.9696159145360694 test: 0.9619800018738477
train: 0.9662463199475176 test: 0.9646021386705141


Log res shows simular scores on all datasets

Random forest:

In [29]:
# Grid param
tuned_parameters = [{ 'n_estimators': [i for i in range (100,300,20)],
                     'max_depth': [70]}]

In [30]:
Models_for=[]
score_for=[]
for i in range(0,3):
    Model2 = GridSearchCV(RandomForestClassifier(),
                      tuned_parameters, cv=5, scoring='f1_weighted',n_jobs=6)
    Model2.fit(set_mat[i][0], set_mat[i][2])
    train_scores=Model2.best_score_
    print(Model2.best_params_)
    Model2=Model2.best_estimator_
    test_scores=metrics.f1_score(y_true=set_mat[i][3],y_pred=Model2.predict(set_mat[i][1]),average='weighted')
    print ('train:',train_scores,'test:',test_scores)
    score_for.append(test_scores)
    Models_for.append(Model2)
scores.append(score_for)
# save models
save_model(Models_for,'Models/RF_model')

{'max_depth': 70, 'n_estimators': 260}
train: 0.9580030216785804 test: 0.9471361343205387
{'max_depth': 70, 'n_estimators': 220}
train: 0.9574553717237544 test: 0.9442651972509771
{'max_depth': 70, 'n_estimators': 280}
train: 0.9515462171353745 test: 0.9373673135135523


So, all results are near to each other, ifidf needs less trees that bags of words. Anyway short clean data set needs more trees, but shows the same perfomance.

xg-boost:

In [31]:
xgb_model = xgb.XGBClassifier()

In [33]:
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['multi:softmax'],
              'learning_rate': [0.05], #so called `eta` value
              'max_depth': [50],
              'min_child_weight': [11],
              'silent': [1],
              'subsample': [0.8],
              'colsample_bytree': [0.7],
              'n_estimators': [i for i in range (100,300,25)],
              'missing':[-999],
              'seed': [1337]}

In [ ]:
# Models_xg=[]
# score_xg=[]
# for i in range(0,3):
#     Model2 = GridSearchCV(xgb_model, parameters, n_jobs=5, 
#                    cv=5,scoring='f1_weighted')
#     Model2.fit(set_mat[i][0], set_mat[i][2])
#     train_scores=Model2.best_score_
#     print(Model2.best_params_)
#     Model2=Model2.best_estimator_
#     test_scores=metrics.f1_score(y_true=set_mat[i][3],y_pred=Model2.predict(set_mat[i][1]),average='weighted')
#     print ('train:',train_scores,'test:',test_scores)
#     score_xg.append(test_scores)
#     Models_xg.append(Model2)
# scores.append(score_xg)
# # save models
# save_model(Models_xg,'XG_model')

In [ ]:
# loaded_model = pickle.load(open(filename, 'rb'))

## CNN

In [35]:
from keras.layers import Input,Dense,Activation,Dropout,BatchNormalization
from keras.models import Model
from keras.utils import np_utils
from keras.optimizers import adam
from keras import regularizers
from sklearn.metrics import classification_report
import tensorflow as tf
from keras import backend as K

Using TensorFlow backend.


In [43]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1) 

In [36]:
def modelFNN(X_train,N,drop):    
    inputs = Input(shape=(X_train.shape[1],))
    x=Dense(N, activation='relu',kernel_regularizer=regularizers.l2(0.0009))(inputs)
    drop=Dropout(drop)(x)
    x=Dense(N, activation='relu',kernel_regularizer=regularizers.l2(0.0008))(drop)
    output_layer = Dense(5, activation = 'softmax')(x)
    model1 = Model(inputs=inputs, outputs=output_layer,)
    ADAM=adam(lr=0.0001)
    model1.compile(optimizer=ADAM,
              loss='categorical_crossentropy',
              metrics=['accuracy', f1])
    return model1

In [37]:
model=modelFNN(X_train,50,0.3)

In [56]:
model.fit(X_train,y_train,batch_size=16,epochs=1,validation_data=(X_val,y_val),verbose=1)

Train on 1341 samples, validate on 149 samples
Epoch 1/1
1341/1341 [==============================] - 1s 865us/step - loss: 0.0695 - acc: 0.9948 - f1: 0.9661 - val_loss: 0.1525 - val_acc: 0.9732 - val_f1: 0.9645


In [63]:
# find optimal parametrs
res=[]
N=[i for i in range(10,200,30)]
drop=np.linspace(0.1,0.7,7)
for i in N:
    print (i)
    line=[]
    for j in drop:
        print ('   -- ',j)
        model=modelFNN(X_train,i,j)
        model.fit(X_train,y_train,batch_size=16,epochs=25,validation_data=(X_val,y_val),verbose=0)
        preds=model.predict(X_test).argmax(1)
        line.append(metrics.f1_score(y_true=y_test.argmax(1),y_pred=preds,average='weighted'))
    res.append(line)
Report=pd.DataFrame(data=res)
Report.columns=drop;Report.index=N

10
   --  0.1
   --  0.2
   --  0.3
   --  0.4
   --  0.5
   --  0.6
   --  0.7
40
   --  0.1
   --  0.2
   --  0.3
   --  0.4
   --  0.5
   --  0.6
   --  0.7
70
   --  0.1
   --  0.2
   --  0.3
   --  0.4
   --  0.5
   --  0.6
   --  0.7
100
   --  0.1
   --  0.2
   --  0.3
   --  0.4
   --  0.5
   --  0.6
   --  0.7
130
   --  0.1
   --  0.2
   --  0.3
   --  0.4
   --  0.5
   --  0.6
   --  0.7
160
   --  0.1
   --  0.2
   --  0.3
   --  0.4
   --  0.5
   --  0.6
   --  0.7
190
   --  0.1
   --  0.2
   --  0.3
   --  0.4
   --  0.5
   --  0.6
   --  0.7


In [66]:
# Report.to_csv('fnn.csv')

In [38]:
# load report 
report=pd.read_csv('fnn.csv')

In [39]:
report

,Unnamed: 0,0.1,0.2,0.3,0.4,0.5,0.6,0.7
0,10,0.961995,0.941710,0.945479,0.950900,0.957926,0.966020,0.968778
1,40,0.970085,0.968721,0.960733,0.961963,0.968735,0.966069,0.956437
2,70,0.966071,0.970081,0.967338,0.967355,0.963309,0.966027,0.970097
3,100,0.965979,0.964637,0.971435,0.970135,0.971468,0.966052,0.964603
4,130,0.965963,0.967396,0.966122,0.970081,0.970058,0.966042,0.967430
5,160,0.963280,0.959275,0.970067,0.972780,0.971419,0.961958,0.968707
6,190,0.965982,0.961990,0.965978,0.954990,0.964625,0.968687,0.968785


Best score parametrs :
test score: 0.972780
drop:0.4 
new:160

datasets perfomans comperasion cnn:

In [40]:
y_cat=np_utils.to_categorical(y,5)
X_b_train, X_b_test, y_b_train, y_b_test = train_test_split(df_bags.iloc[:,:], y_cat, test_size=0.33, random_state=20)
X_tf_train, X_tf_test, y_tf_train, y_tf_test = train_test_split(df_tf.iloc[:,:], y_cat, test_size=0.33, random_state=20)
X_cl_train, X_cl_test, y_cl_train, y_cl_test = train_test_split(df_cl.iloc[:,:], y_cat, test_size=0.33, random_state=20)
set_mat=[[X_b_train, X_b_test, y_b_train, y_b_test],
        [X_tf_train, X_tf_test, y_tf_train, y_tf_test],
        [X_cl_train, X_cl_test, y_cl_train, y_cl_test]]

In [48]:
Models_cnn=[]
score_cnn=[]
for i in range(0,3):
    model=modelFNN(set_mat[i][0],160,0.4)
    model.fit(set_mat[i][0], set_mat[i][2],batch_size=16,epochs=25,verbose=0)
    train_scores = metrics.f1_score(y_true=set_mat[i][2].argmax(1),y_pred=model.predict(set_mat[i][0]).argmax(1),average='weighted')
    test_scores=metrics.f1_score(y_true=set_mat[i][3].argmax(1),y_pred=model.predict(set_mat[i][1]).argmax(1),average='weighted')
    print ('train:',train_scores,'test:',test_scores)
    score_cnn.append(test_scores)
    Models_cnn.append(model)
    # save models
    model.save('Models/cnn'+str(i)+'.h5')
    model.save_weights('Models/cnn_w'+str(i)+'.h5')
scores.append(score_cnn)

train: 1.0 test: 0.9728139720512199
train: 1.0 test: 0.9727913064717708
train: 1.0 test: 0.9645847196452797


## Results

In [71]:
final=pd.DataFrame(data=score,columns=['bag_words','ifidf','Clean_bag'],
                   index=['Log-reg','Random forest','CNN'])


In [72]:
final

,bag_words,ifidf,Clean_bag
Log-reg,0.963219,0.961980,0.964602
Random forest,0.947136,0.944265,0.937367
CNN,0.972814,0.972791,0.964585


The best model is cnn (through the results), shown f1=0.97 on the main bag_words, but if you play with the parameter, you can get the same results for Clean_bag. Summarize that clean_bag is good, of course, it is easier basic and can be used to get close or even the same results as others.